In [ ]:
# Load the python modules.
import os, sys
#Insert the common folder in the path (to be able to load python module)
sys.path.insert(0,os.path.join(os.path.pardir,'common'))
import numpy as np
import wget
from dapper import Chronology

from toolbox import load_config, path, get_filenames, load_data, rmse_norm
from l2s_utils import plot_L96_2D, default_param

# Setting

## Source of the data
The variable ```source``` can be set to ```"ftp"``` or ```"local"```.
- ```ftp```: it will download the data from an ftp server, so there is no need to compute the data. WARNING: it will override existing data in the data simulations data directory.
- ```local```: it will load the data locally. So you have to first either having downloaded the data from the ftp server, either having computing the data using the other notebooks.

## Configuration files
There are 4 configuration file to set. Default configuration files are provided but if you want to make further experiment, it is possible to change the configuration file.
- ```config_path```: contains all the paths where to load and save the data.
- ```config_ftp```: contains the url of the data in the ftp server. This configuration file is optional if you don't download the data from ftp.
- ```config_ref```: contains the reference simulations settings.
- ```config_sens```: contains the different settings for an sensitivity experiment.

In [ ]:
#Source of the data (comment/uncomment the lines given your choice)
source = 'local'
#source = 'ftp'


In [ ]:
#Configuration files
config_path = 'config/paths.yml'
config_ftp = 'config/ftp.yml' #optional if data are generated locally
config_ref = 'config/ref.yml'
config_sens = 'config/sens.yml'

In [ ]:
# Load the paths (yml file)
paths = load_config('config/paths.yml')

# Rootdir for the L2S data
rootdir = path(paths['rootdir'])

#Check if figdir exist and set the figdir accordingly
savefig = 'figdir' in paths and bool(paths['figdir'])
figdir = path(os.path.join(rootdir,paths['figdir'])) if savefig else None

#Directory containing the simulations
testdir = path(os.path.join(rootdir,paths['testdir']))

#Print:
print("Directory containing the simulations:\n->",testdir)
if figdir:
    print("Directory to save the figures:\n->",figdir)

In [ ]:
#Load the ftp configuration file if needed
if source == 'ftp':
    ftp = load_config(config_ftp)
    ftpurl, ftpdir = ftp['url'], ftp['test']
    print('\033[93m'+'Warning! The source is set to ftp. \nIt will override existing local simulation in the simulation directory'+'\033[0m')
else:
    ftp = None
    ftpurl, ftpdir = None, None

## Plot Figure 1

In [ ]:
# Define the reference experiment to plot
ref = load_config(config_ref)
print('Parameters of the reference experiment:\n',ref['params'])
print('Template names used to load the reference experiment:\n',ref['files'])

In [ ]:
# Load the data of the hybrid/true run
filename = get_filenames(ref, default_param=default_param)

#Check if there is exactly one filename to load
assert len(filename['hybrid']) == 1

#Set the filename (as a str) of the data
fname = str(filename['truth'].name.iloc[0])
data_truth = load_data(testdir,fname,ftpurl=ftpurl,ftpdir=ftpdir)

print("Data file truth:\n->",fname)

fname = str(filename['hybrid'].name.iloc[0])
data_hybrid = load_data(testdir,fname,ftpurl=ftpurl,ftpdir=ftpdir)

print("Data file hybrid:\n->",fname)

In [ ]:
# Format the data (the time step of the hybrid model is different from the time step of the real model)

#hybrid data
xx_hybrid = data_hybrid['xx']

#chronology
chrono_hybrid = Chronology(dkObs=1,T=data_hybrid['T'],dt=data_hybrid['dt'])
chrono_truth = Chronology(dkObs=1,T=data_truth['T'],dt=data_truth['dt'])

#Compute the time step subsample:
dk = int(chrono_hybrid.dt/chrono_truth.dt)

#Compute the size of the hybrid model stat:
nU = xx_hybrid.shape[-1]

#Subsampe the true model in space and time to fit the hybrid model space
xx_truth = data_truth['xx'][::dk,:,:nU]

#Define the time axis
tt = chrono_hybrid.tt

#Display the size of the model simulations
print('Size of the time axis:',tt.shape)
print('Size of the truth simulation (time step, members, space):',xx_truth.shape)
print('Size of the hybrid simulation (time step, members, space):',xx_hybrid.shape)

In [ ]:
# Plot the figure 1

#Ensemble member to plot:
iens=1

#Number of time steps to plot
limT = 500

#Plot
fig = plot_L96_2D(xx_truth[:limT,iens],xx_hybrid[:limT,iens],tt[:limT],['Truth','NN hyb.'])

#Save (if needed)
if figdir:
    fig.savefig(os.path.join(figdir,'philtra-fig1.png'),
            dpi=200, bbox_inches='tight', pad_inches=0)

## Plot Figure 2

In [ ]:
sens = load_config(config_sens)
print('Parameters of the sensitivity experiment:\n',sens['params'])
print('Template names used to load the reference experiment:\n',sens['files'])

#Reload ref file (in case you haven't been gone through Plot Figure 1)
ref = load_config(config_ref)
ref_params = ref['params']

In [ ]:
# (Re)Load the data of the true/truncated run
filename = get_filenames(sens, default_param=default_param)

#Check if there is exactly one filename to load
assert len(filename['truth']) == 1

if source == 'ftp':
    ftpurl, ftpdir = ftp['url'], ftp['test']
else:
    ftpurl, ftpdir = None, None

#Set the filename (as a str) of the data
fname = str(filename['truth'].name.iloc[0])
data_truth = load_data(testdir,fname,ftpurl=ftpurl,ftpdir=ftpdir)

print("Data file truth:\n->",fname)

#Set the filename (as a str) of the data
fname = str(filename['trunc'].name.iloc[0])
data_trunc = load_data(testdir,fname,ftpurl=ftpurl,ftpdir=ftpdir)

print("Data file trunc:\n->",fname)

#Set the filename (as a str) of the data
fname = str(filename['perfect'].name.iloc[0])
data_perfect = load_data(testdir,fname,ftpurl=ftpurl,ftpdir=ftpdir)

print("Data file perfect:\n->",fname)

In [ ]:
# Format the data (the time step of the trunc model is different from the time step of the real model)


#Trunc data
xx_trunc = data_trunc['xx']

#chronology
chrono_trunc = Chronology(dkObs=1,T=data_trunc['T'],dt=data_trunc['dt'])
chrono_truth = Chronology(dkObs=1,T=data_truth['T'],dt=data_truth['dt'])

#Compute the time step subsample:
dk = int(chrono_trunc.dt/chrono_truth.dt)

#Compute the size of the trunc model stat:
nU = xx_trunc.shape[-1]

#Subsampe the true model in space and time to fit the trunc model space
xx_truth = data_truth['xx'][::dk,:,:nU]

#Define the time axis
tt = chrono_trunc.tt

#Display the size of the model simulations
print('Size of the time axis:',tt.shape)
print('Size of the truth simulation (time step, members, space):',xx_truth.shape)
print('Size of the trunc simulation (time step, members, space):',xx_trunc.shape)

In [ ]:
#Reformat DataFrame
import pandas as pd

#Add a column in the Data frame with the type of simulation
for k in filename:
    filename[k]=filename[k].assign(model=k)

#Concatenate all the dataframe in one:
df = pd.concat([v for k,v in filename.items()], ignore_index=True)
df

In [ ]:
#Compute the RMSE os all the  simulations
# If ftp is True, it can take some few minutes
RMSE = dict()

#Time to compute the RMSE
it = 200

df.assign(RMSE = np.nan) #Default value (changed in the loop)
print('->compute RMSE for the all the models (except the truth)')
for idx in df[df.model != 'truth'].index:
    fname = df.loc[idx,'name']
    data = load_data(testdir,fname,ftpurl=ftpurl,ftpdir=ftpdir)
    xx_est = data['xx']
    RMSE[idx] = rmse_norm(xx_truth,xx_est)
    df.loc[idx,'RMSE'] = RMSE[idx][it]
    data.close()


In [ ]:
#Initialise the figure
import matplotlib.pyplot as plt
fig, (ax, ax0,ax1) = plt.subplots(ncols=3,figsize=(8.1,3),sharey='all')


#Plot the sensitivity to dtobs
df_dthyb = df[(df['Nfil_train']==ref_params['Nfil_train'][0])&
          (df['std_o']==ref_params['std_o'][0])&
          (df['p']==ref_params['p'][0]) & (df['model']=='hybrid')
         ]

df_dtpo = df[df['model']=='perfect']
df_trunc = df[df['model']=='trunc']

x = np.arange(df_dthyb.shape[0])
xtrunc = [len(x)]
ax0.bar(x=x-0.2,height=df_dthyb['RMSE'],width=0.4,color='C1',label='DA hyb.');
ax0.bar(x=x+0.2,height=df_dtpo['RMSE'],width=0.4,color='C2',label='perf.obs. hyb.');
ax0.bar(x=xtrunc,height=df_trunc['RMSE'],width=0.4,color='C3',label='trunc.')
ax0.set_xticks(list(x)+xtrunc);
ax0.set_xticklabels([str(dt) for dt in df_dthyb['dtObs']]+['trunc.']);
ax0.bar(x=x[1]-0.2,height=df_dthyb.iloc[1]['RMSE'],width=0.4,fill=False,LineWidth=2,edgecolor='black',label='ref.exp.')

ax0.legend()
ax0.set_xlabel('$\Delta t$')

#Plot the sensitivity to the filtration
df_filhyb = df_stdo = df[(df['dtObs']==ref_params['dtObs'][0]) & 
          (df['p']==ref_params['p'][0]) &
          (df['Nfil_train'].isin([1,5])) & (df['model']=='hybrid')
         ]
df_1 = df_filhyb[df_filhyb['Nfil_train']==1]
df_5 = df_filhyb[df_filhyb['Nfil_train']==5]


x = np.arange(df_1.shape[0])
ax1.bar(x=x-0.2,height=df_1['RMSE'],
        width=0.4,color='C1',label='no filter');
ax1.bar(x=x+0.2,height=df_5['RMSE'],
        width=0.4,color='C4',label='filter 0.05 MTU');
ax1.bar(x=x[1]-0.2,height=df_1.iloc[1]['RMSE'],
        width=0.4,fill=False,LineWidth=2,edgecolor='black',label='ref.exp.')
ax1.set_xticks(x);
ax1.set_xticklabels(['{}'.format(stdo) for stdo in df_1['std_o']]);
ax1.set_xlabel('$\sigma^o$')
ax1.legend()

#Plot the sensitivity to the density of observations (time series)
#Plot the sensitivity to dtobs

#Number time step to display
limT = 800

#Displayed color
cols=['C5','C6','C1']

df_phyb = df[(df['Nfil_train']==ref_params['Nfil_train'][0])&
          (df['std_o']==ref_params['std_o'][0])&
          (df['dtObs']==ref_params['dtObs'][0]) & 
             (df['model']=='hybrid')
         ]

df_ppo = df[(df['model']=='perfect') & 
            (df['dtObs']==ref_params['dtObs'][0])]

for i,idx in enumerate(df_phyb.index):
    ax.plot(tt[:limT],RMSE[idx][:limT],
            label='$N_y$={}'.format(df_phyb.loc[idx,'p']),
           color=cols[i])
ax.plot(tt[:limT],RMSE[df_trunc.index[0]][:limT],
        label='trunc.',color='C3',linewidth=2)

ax.plot(tt[:limT],RMSE[df_ppo.index[0]][:limT],
        label='perf. obs.',color='C2',linewidth=2)

ax.legend();

if figdir:
    fig.savefig(os.path.join(figdir,'philtra-fig2.png'),
            dpi=200, bbox_inches='tight', pad_inches=0)
